In [1]:
import langchain
langchain.__version__

'0.3.20'

In [2]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [9]:
from langchain_community.llms import Ollama

# Use the correct model name from your Ollama server
llm = Ollama(
    model="gemma3:latest",  # Correct model name from API response
    base_url="http://127.0.0.1:11434",  # Ensure it's pointing to the right Ollama instance
    temperature=0.5
)

# Test the model
#response = llm.invoke("Explain the benefits of AI in healthcare.")
#print(response)


In [22]:
from langchain.agents import initialize_agent, Tool, create_sql_agent
from dotenv import load_dotenv
import os

In [23]:
# Using duckduckgo for web search
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
#Web Search Tool using DuckDuckGo
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo."
)
tools = [search_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("What are the top trending movies this week?")
print(response)



> Entering new AgentExecutor chain...
I need to search the web to find out what movies are trending currently. Using DuckDuckGo Search will help me gather the information.
Action: DuckDuck
Action Input: 'What are the trending movies this week?'
Observation: DuckDuck is not a valid tool, try one of [DuckDuckGo Search].
Thought:I am sorry, but DuckDuckGo Search is the only tool available to me. I will use it to search for trending movies this week.
Action: DuckDuckGo Search
Action Input: 'Trending movies this week'
Observation: Watch the latest movies and TV shows via Sling. Score your first 3 months for $99.99, $140 off. Channels available include ABC, NBC, and Fox as well as ESPN, Bravo, FX, National Geographic, and ... I've pored over release schedules to bring you the best original and new-to-streaming movies you can watch on Netflix, Prime, Max, Hulu, and other streaming platforms this week. My don't-miss-it ... The Day the Earth Blew Up: A Looney Tunes Movie PG Porky and Daffy ar

In [14]:
# The WikipediaAPIWrapper - allows you to query Wikipedia for articles and summaries.

!pip install wikipedia

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Step 1: Initialize the Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic."
)

tools = [wikipedia_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

query = "Who is Alan Turing, and what are his contributions?"
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
Alan Turing is a hugely significant figure in computer science and artificial intelligence. I should use Wikipedia to understand his life and his key contributions.
Action: Wikipedia Search
Action Input: “Alan Turing”
Observation: Page: Alan Turing
Summary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science.
Born in London, Turing was raised in southern England. He graduated from King's College, Cambridge, and in 1938, earned a doctorate degree from Princeton University. During World War II, Turing worked for the Government Co

In [24]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")

Database created and populated with sample data.


In [25]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [28]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example.db")

# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent="zero-shot-react-description",
#     verbose=True,
#     handle_parsing_errors=True
# )

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_orders
Action Input:sql_db_list_orders is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].sql_db_list_tables is the most appropriate tool here as it can help me understand the database schema.
Action: sql_db_list_tables
Action Input:inventoryOkay, I have the table names: `inventory`. Now I need to query this table to find the number of units of Product A in stock.
Action: sql_db_query
Action Input: SELECT CAST(units AS INTEGER) FROM inventory WHERE product_id = 'A';
Error: (sqlite3.OperationalError) no such column: units
[SQL: SELECT CAST(units AS INTEGER) FROM inventory WHERE product_id = 'A';
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)The error states "no such column: units", which suggests that the column name might be different in the `inventory` table. I should use sql_db_schema to check the table schema and find the correct column name.
Action: 

In [34]:
# ArxivQueryRun
#!pip install arxiv
from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent
query = "Search for latest research papers on Generative Adversarial Network (GAN)."
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
Okay, let's use the Arxiv Search tool to get the latest research papers on Generativie Adversarial Networks (GANs). I'll use the tool to find papers published in the past month that discuss GANs.
Action: Arxive Search
Action Input: copy of the following dictionary: `{`"topics": "GAN"`}`
Observation: Arxive Search is not a valid tool, try one of [Arxiv Search].
Thought:Okay, I apologize for the mistake. I need to use the `ArXiv Search` tool properly. Let’s try a different search term to target GAN papers.
Action: ArXiv Search
Action Input: `{}`
Thought: I need to be more specific in my search to find relevant GAN papers. I will broaden my search to include "Generative Adversarial Networks" and specify the publication date.
Observation: ArXiv Search is not a valid tool, try one of [Arxiv Search].
Thought:I apologize for the repeated errors. I need to ensure I'm using the correct tool. The tool is `Arxiv Search`. Let's proceed with a more focused se

In [41]:
import os
print(os.getcwd())  # This shows your current working directory
print(os.listdir())  # This lists all files in that directory

/Users/srinivasraomarri/Documents/AI-ML/agents
['.DS_Store', 'example.db', 'agents.pdf', 'kubernetes-langgraph-bot', 'example.txt', '.ipynb_checkpoints', 'langchain.ipynb', 'langchain-venv']


In [44]:
from langchain_community.tools.file_management import ReadFileTool

# Step 1: Initialize the ReadFileTool
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

# Step 3: Combine the tool into a list
tools = [read_file_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

# Step 5: Query the Agent with a file name

#file_name = "/Users/srinivasraomarri/Documents/AI-ML/agents/example.txt"
# http://localhost:8888/files/example.txt?_xsrf=2%7Ce9c7f29e%7Cf4cc64880f58f72e4d1d7ebe38558746%7C1739824459
file_name="/Users/srinivasraomarri/Documents/AI-ML/agents/example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
I need to read the file at /Users/sarinivasraomarri/Documents/AI-ML/agents/example.txt and return its content. I'll use the Read File tool for this purpose.
Action: Read File
Action Input: example.txt
Observation: Hello agents.
Thought:Now I have the content of the file.
Final Answer: Hello agents.

> Finished chain.
Response from Agent:
Hello agents.
